# Calculating Sesonality Index on weekly bases

### Inserting data and checking dtypes



In [ ]:
# import what needed
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# adding path
path = r'SEAS.xlsx'

# creatiung dataframe
df = pd.read_excel(path)

# checking types of variables
print(df.dtypes)

df.head()

### Standardizing Variable Names for Consistency

In [ ]:
# fixing variables names for consistenly and removing white spaces if any
df.columns = df.columns.str.lower().str.strip().str.replace(' ', '_')

df.columns

### Adding new columns in order to track seasonality and calculate seasonality on weekly bases

In [36]:
# Ensure 'date' column is in datetime format
df['date'] = pd.to_datetime(df['date'], errors='coerce')

# Extract the week number before dropping columns with NaNs or zeros
df['week'] = df['date'].dt.isocalendar().week

# Drop columns that are entirely NaN
df = df.dropna(axis=1, how='all')

# Drop columns that contain only zeros
df = df.loc[:, (df != 0).any(axis=0)]
df.columns
# # Calculate 'total' by summing numeric columns except 'date' and 'week'
# df['total'] = df.loc[:, ~df.columns.isin(['date', 'week'])].sum(axis=1, numeric_only=True)

# # Plotting
# sns.relplot(data=df, kind='line', x='week', y='total')
# plt.show()


Index([                           'date', 'inbound_unloading_+_palletizing',
                         'unloading_goh',                     'putaway_goh',
                 'unloading_consumables',              '%_check_on_inbound',
                'troubleshooter_inbound',           'inbound_waiting_hours',
                    'masterdata_new_sku',           'opt_putaway_+_picking',
                         'opt_assigners',                               nan,
                            'opt_intern',   'piece_picking_mezzanine_netto',
                           'picking_goh',     'putaway_after_replenishment',
                'troubleshooter_picking',               'picking_randzaken',
                        'cycle_counting',                   'stock_control',
                         'sorter_infeed',            'sorter_packing_netto',
                'sorter_trouble_shooter',            'sorter_folding_boxes',
                  'sorter_destroy_boxes',                    'sorter_aorta',